In [1]:
# =========================
# Library
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
from joblib import Parallel, delayed
import Levenshtein
import difflib
from contextlib import contextmanager
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss
from strsimpy import SIFT4
from strsimpy.ngram import NGram
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# =========================
# Constant
# =========================
TRAIN_PATH = "../data/train.csv"
TARGET = "point_of_interest"

In [3]:
# =========================
# Settings
# =========================
fe = "069"
if not os.path.exists(f"../output/fe/fe{fe}"):
    os.makedirs(f"../output/fe/fe{fe}")

In [5]:
# =========================
# Main
# =========================
train = pd.read_pickle("../output/fe/fe066.pkl") 
train_raw = pd.read_csv(TRAIN_PATH)
train_raw = train_raw[["id","name"]].reset_index(drop=True)

In [7]:
train_raw["name"] = train_raw["name"].astype(str)
train_raw["name"] = train_raw["name"].str.lower()

In [8]:
id_unique = train_raw["id"].values
id2name_dict = {}
for i,n in zip(train_raw["id"].values,train_raw["name"].values):
    id2name_dict[i] = n

In [9]:
train["name_lower"] = train["id"].map(id2name_dict)
train["near_name_lower"] = train["near_id"].map(id2name_dict)

In [10]:
emb = np.load("../output/fe/fe029/bert_base_multilingual_embedding.npy")
name = np.load("../output/fe/fe029/name.npy",allow_pickle=True)

In [11]:
with open(f'../output/fe/fe046/fe046_svd.pkl','rb') as f:
    svd = pickle.load(f)
emb_svd = svd.transform(emb)

In [12]:
name_svd = np.zeros([len(train),10],np.float32)
near_name_svd = np.zeros([len(train),10],np.float32)

In [13]:
name2num_dict = {}
for n,i in enumerate(name):
    name2num_dict[i] = n

In [14]:
train["name_le"] = train["name_lower"].map(name2num_dict)

In [15]:
train["near_name_le"] = train["near_name_lower"].map(name2num_dict)

In [18]:
for n,i in tqdm(enumerate(train["name_le"].values)):
    name_svd[n,:] = emb_svd[i,:]
for n,i in tqdm(enumerate(train["near_name_le"].values)):
    near_name_svd[n,:] = emb_svd[i,:]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [19]:
name_svd = pd.DataFrame(name_svd)
name_svd.columns = [f"name_emb_svd{i}" for i in range(10)]
near_name_svd = pd.DataFrame(near_name_svd)
near_name_svd.columns = [f"near_name_emb_svd{i}" for i in range(10)]
name_svd = pd.concat([name_svd,near_name_svd],axis=1)

In [21]:
new_cols = ['name_emb_svd0', 'name_emb_svd1', 'name_emb_svd2', 'name_emb_svd3',
       'name_emb_svd4', 'name_emb_svd5', 'name_emb_svd6', 'name_emb_svd7',
       'name_emb_svd8', 'name_emb_svd9', 'near_name_emb_svd0',
       'near_name_emb_svd1', 'near_name_emb_svd2', 'near_name_emb_svd3',
       'near_name_emb_svd4', 'near_name_emb_svd5', 'near_name_emb_svd6',
       'near_name_emb_svd7', 'near_name_emb_svd8', 'near_name_emb_svd9']

In [22]:
name_svd = reduce_mem_usage(name_svd)
name_svd.to_pickle(f"../output/fe/fe{fe}.pkl")

Memory usage of dataframe is 3318.34 MB
column =  20
0
Memory usage after optimization is: 3318.34 MB
Decreased by 0.0%
